**This notebook shows how to perform read & write operations on Interactive Map Layers.**

<div class="alert alert-block alert-warning">
<b>Warning:</b> Before running below cells please make sure you have access to HERE platform.
                For credentials setup follow below steps:
                <ol>
                  <li>Go to <a href="https://platform.here.com/profile/apps-and-keys">HERE Platform Applications and Keys</a> and register a new app.</li>
                  <li>Create a key for the app and download the generated credentials.properties file</li>
                  <li>copy values from credentials.properties file into environment variables HERE_USER_ID, HERE_CLIENT_ID, HERE_ACCESS_KEY_ID, HERE_ACCESS_KEY_SECRET
HERE_TOKEN_ENDPOINT_URL</li>
                </ol>
                Please check this <a href="https://developer.here.com/documentation/identity-access-management/dev_guide/index.html">link</a> for more information on platform authentication.
</div>

In [ ]:
# uncomment the below cell and replace actual values from credentials.properties file

In [ ]:
# import os

# os.environ["HERE_USER_ID"] = "Replace value here from credentials.properties file"
# os.environ["HERE_CLIENT_ID"] = "Replace value here from credentials.properties file"
# os.environ["HERE_ACCESS_KEY_ID"] = "Replace value here from credentials.properties file"
# os.environ["HERE_ACCESS_KEY_SECRET"] = "Replace value here from credentials.properties file"
# os.environ["HERE_TOKEN_ENDPOINT_URL"] = "Replace value here from credentials.properties file"

In [ ]:
from xyzspaces import IML
from xyzspaces.iml.credentials import Credentials

## Create a new catalog and interactive map layer

In [ ]:
# Details of the new interactive map layer.

layer_details = {
    "id": "countries",
    "name": "countries",
    "summary": "Borders of world countries.",
    "description": "Borders of world countries",
    "layerType": "interactivemap",
    "interactiveMapProperties": {},
}

In [ ]:
# Create a new interactive map layer and catalog.

credentials = Credentials.from_env()

iml = IML.new(
    catalog_id="test-catalog-iml-demo",
    catalog_name="test-catalog-iml-demo",
    catalog_summary="This is catalog for demo",
    catalog_description="This is catalog for demo.",
    layer_details=layer_details,
    credentials=credentials,
)

In [ ]:
iml.layer

<div class="alert alert-block alert-warning">
<b>Warning:</b> Before running below cell please make sure you have API key. 
                Please see <a href="https://github.com/heremaps/here-map-widget-for-jupyter/blob/master/examples/README.md">README.md</a> for more info on API key.
</div>

In [ ]:
import os

os.environ["LS_API_KEY"] = "YOUR-API-KEY"  # replace your api key here.

In [ ]:
from here_map_widget import Map, GeoJSON


def get_map(data, zoom=2, center=(0, 0)):
    m = Map(api_key=os.environ["LS_API_KEY"], zoom=zoom, center=center)
    geojson = GeoJSON(data=data)
    m.add_layer(geojson)
    return m

In [ ]:
import requests

countries = requests.get(
    "https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json"
).json()

In [ ]:
m = get_map(data=countries)
m

## Write countries data to interactive map layer.

In [ ]:
iml.layer.write_features(features=countries)

## Check statistics of the layer

In [ ]:
iml.layer.statistics

## Get single feature from interactive map layer.

In [ ]:
resp = iml.layer.get_feature(feature_id="IND")
feature = resp.to_geojson()

In [ ]:
m = get_map(data=feature)
m

## Get multiple features from interactive map layer.

In [ ]:
resp = iml.layer.get_features(feature_ids=["IND", "USA", "DEU"])
features = resp.to_geojson()

In [ ]:
m = get_map(data=features, center=(33.3965, 2.3222))
m

## Iterate over & inspect all features in a layer

In [ ]:
from geojson import FeatureCollection

features = list(iml.layer.iter_features())
fc = FeatureCollection(features=features)
m = get_map(data=fc, center=(33.3965, 2.3222))
m

## Search features from layer based on their properties

In [ ]:
resp = iml.layer.search_features(
    params={"p.name": ["India", "United States of America", "Australia", "Germany"]}
)
searched_features = resp.to_geojson()
m = get_map(data=searched_features, center=(33.3965, 2.3222))
m

## Get features in a bounding box

In [ ]:
bbox = (68.1766451354, 7.96553477623, 97.4025614766, 35.4940095078)

resp = iml.layer.get_features_in_bounding_box(bbox)
bbox_features = resp.to_geojson()

m = get_map(data=bbox_features, center=(33.3965, 2.3222))
m

## Spatial search within radius

In [ ]:
resp = iml.layer.spatial_search(lat=0, lng=30, radius=1000000)

radius_search = resp.to_geojson()


m = get_map(data=radius_search, center=(33.3965, 2.3222))
m

## Spatial search using Geometry Intersection

In [ ]:
from geojson import Point, Feature

pt = Point((73, 19))
resp = iml.layer.spatial_search_geometry(geometry=pt)
geometry_search = resp.to_geojson()

m = get_map(data=geometry_search, center=(33.3965, 2.3222))
m

In [ ]:
# Delete the catalog

iml.delete_catalog(
    catalog_hrn="hrn:here:data::olp-here:test-catalog-iml-demo", credentials=credentials
)